<a href="https://colab.research.google.com/github/Demi-greaterme/ML_Bearing-Failure/blob/main/Predictive_Analysis_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing necessary packages
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew

✅ Done! 157 total rows generated.
   mean_AcX   std_AcX   rms_AcX  kurt_AcX  skew_AcX  crest_AcX  mean_AcY  \
0  0.014925  0.002160  0.015081 -1.249248 -0.533569   1.137285  0.006145   
1  0.014212  0.001909  0.014339 -1.293685 -0.206163   1.187558  0.006037   
2  0.015217  0.002231  0.015379 -1.080564 -0.799624   1.119149  0.006042   
3  0.015900  0.001419  0.015963 -0.131267 -1.034465   1.085893  0.006212   
4  0.014215  0.001855  0.014336 -1.303053 -0.253051   1.170808  0.006060   

    std_AcY   rms_AcY  kurt_AcY  skew_AcY  crest_AcY  mean_AcZ   std_AcZ  \
0  0.000211  0.006149  0.419805 -1.253483   1.042216  0.000301  0.000094   
1  0.000233  0.006041 -1.186562 -0.466566   1.050721  0.000280  0.000101   
2  0.000229  0.006047 -1.169126 -0.469334   1.049752  0.000225  0.000096   
3  0.000071  0.006213  1.391810 -1.108089   1.011908  0.000275  0.000108   
4  0.000199  0.006063 -1.452558 -0.326847   1.046964  0.000299  0.000089   

    rms_AcZ  kurt_AcZ  skew_AcZ  crest_AcZ  label  


In [ ]:
# Loading CSVs containing bearing data
normal = pd.read_csv('/content/Full Load Normal Bearing (1).csv')
fault  = pd.read_csv('/content/Full Load Faulty Bearing (1).csv')

In [ ]:
# Time stamp column not needed, dropping
if 'Time ' in normal.columns:
    normal = normal.drop(columns=['time stamp'])
if 'Time' in fault.columns:
    fault = fault.drop(columns=['time stamp'])

In [ ]:
# Feature extraction for one segment (1 second)
def extract_features(df):
    feats = {}
    for axis in ['AcX', 'AcY', 'AcZ']:
        data = df[axis].values
        feats[f'mean_{axis}'] = np.mean(data)
        feats[f'std_{axis}'] = np.std(data)
        feats[f'rms_{axis}'] = np.sqrt(np.mean(np.square(data)))
        feats[f'kurt_{axis}'] = kurtosis(data)
        feats[f'skew_{axis}'] = skew(data)
        if feats[f'rms_{axis}'] == 0:
            feats[f'crest_{axis}'] = np.nan
        else:
            feats[f'crest_{axis}'] = np.max(np.abs(data)) / feats[f'rms_{axis}']
    return feats

In [ ]:
# Splitting each file into 1-second segments
def process_file(df, label):
    rows = []
    segment_size = 32
    for start in range(0, len(df), segment_size):
        seg = df.iloc[start:start+segment_size]
        if len(seg) < segment_size:
            continue
        f = extract_features(seg)
        f['label'] = label
        rows.append(f)
    return pd.DataFrame(rows)

In [ ]:
# Merging both datasets into one file
normal_features = process_file(normal, 0)
fault_features  = process_file(fault, 1)

final_df = pd.concat([normal_features, fault_features], ignore_index=True)
final_df.to_csv('bearing_features.csv', index=False)

In [ ]:
print(len(final_df))
print(final_df.head())
final_df.info()
final_df.shape[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mean_AcX   4 non-null      float64
 1   std_AcX    4 non-null      float64
 2   rms_AcX    4 non-null      float64
 3   kurt_AcX   4 non-null      float64
 4   skew_AcX   4 non-null      float64
 5   crest_AcX  4 non-null      float64
 6   mean_AcY   4 non-null      float64
 7   std_AcY    4 non-null      float64
 8   rms_AcY    4 non-null      float64
 9   kurt_AcY   4 non-null      float64
 10  skew_AcY   4 non-null      float64
 11  crest_AcY  4 non-null      float64
 12  mean_AcZ   4 non-null      float64
 13  std_AcZ    4 non-null      float64
 14  rms_AcZ    4 non-null      float64
 15  kurt_AcZ   4 non-null      float64
 16  skew_AcZ   4 non-null      float64
 17  crest_AcZ  4 non-null      float64
 18  label      4 non-null      int64  
dtypes: float64(18), int64(1)
memory usage: 740.0 bytes
